In [15]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [16]:
# read in the streets.json file
streets = gpd.read_file('data/streets.json')
#select only streets_edge_id adn geometry
streets = streets[['street_edge_id', 'geometry']]
streets


,street_edge_id,geometry
0,87,"LINESTRING (-74.03996 40.94794, -74.04089 40.9..."
1,184,"LINESTRING (-74.03088 40.95201, -74.03136 40.9..."
2,477,"LINESTRING (-74.03435 40.95815, -74.03432 40.9..."
3,273,"LINESTRING (-74.03670 40.94918, -74.03755 40.9..."
4,51,"LINESTRING (-74.02456 40.94985, -74.02463 40.9..."
...,...,...
484,465,"LINESTRING (-74.02582 40.94882, -74.02580 40.9..."
485,449,"LINESTRING (-74.03218 40.95121, -74.03216 40.9..."
486,64,"LINESTRING (-74.02885 40.94212, -74.02929 40.9..."
487,55,"LINESTRING (-74.03680 40.95124, -74.03641 40.9..."


In [17]:
#read in labels
labels = gpd.read_file('data/labels.json')
labels.head()
#drop avg_image_capture_data and avg_label_date
labels = labels.drop(['avg_image_capture_date', 'avg_label_date'], axis=1)

In [18]:
#select all missing curb ramps
missing_curb_ramps = labels[labels['label_type'] == 'NoCurbRamp']

In [19]:
#group by street_edge_id and count the number of missing curb ramps
missing_curb_ramps = missing_curb_ramps.groupby('street_edge_id').count()
missing_curb_ramps = missing_curb_ramps[['label_type']]
missing_curb_ramps = missing_curb_ramps.rename(columns={'label_type': 'no_curbs_count'})
missing_curb_ramps = missing_curb_ramps.reset_index()

In [20]:
# sort by count
missing_curb_ramps = missing_curb_ramps.sort_values(by=['no_curbs_count'], ascending=False)
missing_curb_ramps

,street_edge_id,no_curbs_count
169,396,9
139,346,8
29,94,8
1,3,8
50,136,7
...,...,...
43,125,1
118,297,1
117,290,1
116,289,1


In [21]:
#replace > 4 with 4
missing_curb_ramps['no_curbs_count'] = missing_curb_ramps['no_curbs_count'].apply(lambda x: 4 if x > 4 else x)
missing_curb_ramps

,street_edge_id,no_curbs_count
169,396,4
139,346,4
29,94,4
1,3,4
50,136,4
...,...,...
43,125,1
118,297,1
117,290,1
116,289,1


In [38]:
base = missing_curb_ramps_m.explore( tiles = "CartoDB positron")

In [22]:
# merge the streets and missing curb ramps dataframes
streets = streets.merge(missing_curb_ramps, on='street_edge_id', how='left')
streets

,street_edge_id,geometry,no_curbs_count
0,87,"LINESTRING (-74.03996 40.94794, -74.04089 40.9...",NaN
1,184,"LINESTRING (-74.03088 40.95201, -74.03136 40.9...",NaN
2,477,"LINESTRING (-74.03435 40.95815, -74.03432 40.9...",1.0
3,273,"LINESTRING (-74.03670 40.94918, -74.03755 40.9...",NaN
4,51,"LINESTRING (-74.02456 40.94985, -74.02463 40.9...",NaN
...,...,...,...
484,465,"LINESTRING (-74.02582 40.94882, -74.02580 40.9...",NaN
485,449,"LINESTRING (-74.03218 40.95121, -74.03216 40.9...",NaN
486,64,"LINESTRING (-74.02885 40.94212, -74.02929 40.9...",NaN
487,55,"LINESTRING (-74.03680 40.95124, -74.03641 40.9...",NaN


In [23]:
#replace NaN with 0
streets['no_curbs_count'] = streets['no_curbs_count'].fillna(0)
streets

,street_edge_id,geometry,no_curbs_count
0,87,"LINESTRING (-74.03996 40.94794, -74.04089 40.9...",0.0
1,184,"LINESTRING (-74.03088 40.95201, -74.03136 40.9...",0.0
2,477,"LINESTRING (-74.03435 40.95815, -74.03432 40.9...",1.0
3,273,"LINESTRING (-74.03670 40.94918, -74.03755 40.9...",0.0
4,51,"LINESTRING (-74.02456 40.94985, -74.02463 40.9...",0.0
...,...,...,...
484,465,"LINESTRING (-74.02582 40.94882, -74.02580 40.9...",0.0
485,449,"LINESTRING (-74.03218 40.95121, -74.03216 40.9...",0.0
486,64,"LINESTRING (-74.02885 40.94212, -74.02929 40.9...",0.0
487,55,"LINESTRING (-74.03680 40.95124, -74.03641 40.9...",0.0


In [24]:
# delete no_curbs_count = 0
s_a = streets[streets['no_curbs_count'] != 0]
s_b = streets[streets['no_curbs_count'] == 0]

In [31]:
streets
#re

,street_edge_id,geometry,no_curbs_count
0,87,"LINESTRING (-74.03996 40.94794, -74.04089 40.9...",0.0
1,184,"LINESTRING (-74.03088 40.95201, -74.03136 40.9...",0.0
2,477,"LINESTRING (-74.03435 40.95815, -74.03432 40.9...",1.0
3,273,"LINESTRING (-74.03670 40.94918, -74.03755 40.9...",0.0
4,51,"LINESTRING (-74.02456 40.94985, -74.02463 40.9...",0.0
...,...,...,...
484,465,"LINESTRING (-74.02582 40.94882, -74.02580 40.9...",0.0
485,449,"LINESTRING (-74.03218 40.95121, -74.03216 40.9...",0.0
486,64,"LINESTRING (-74.02885 40.94212, -74.02929 40.9...",0.0
487,55,"LINESTRING (-74.03680 40.95124, -74.03641 40.9...",0.0


In [33]:
# eport to geojson
s_a.to_file('data/streets_full.geojson', driver='GeoJSON')

In [71]:
s_a.explore( m= base,column = 'no_curbs_count',legend=True, scheme = 'naturalbreaks', tiles = 'cartodb dark matter', cmap = 'RdPu')

In [13]:
#single out street edge id 389 
s_a_389 = s_a[streets['street_edge_id'] == 389]
s_a_389


,street_edge_id,geometry,no_curbs_count
381,389,"LINESTRING (-74.04580 40.95801, -74.04685 40.9...",4.0


In [77]:
#select street edge id 389 from missing curb ramps
missing_curb_ramps_389 = missing_curb_ramps_m[missing_curb_ramps_m['street_edge_id'] == 389]
missing_curb_ramps_389



,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,agree_count,disagree_count,notsure_count,geometry
1335,814647,NoCurbRamp,389,61313120,West Oradell,3.0,False,2,0,0,POINT (-74.04594 40.95799)
4601,814666,NoCurbRamp,389,61313120,West Oradell,3.0,False,2,0,0,POINT (-74.04917 40.95879)
4626,814641,NoCurbRamp,389,61313120,West Oradell,5.0,False,3,1,0,POINT (-74.04589 40.95809)
5396,814606,NoCurbRamp,389,61313120,West Oradell,4.0,False,1,1,1,POINT (-74.04910 40.95893)


In [79]:
#export missing curb ramps 389 to json
missing_curb_ramps_389.to_file('data/points.geojson', driver='GeoJSON')


In [29]:
missing_curb_ramps_m

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,agree_count,disagree_count,notsure_count,geometry
31,818034,NoCurbRamp,277,11585600,Central Oradell,5.0,False,3,1,1,POINT (-74.03561 40.95972)
32,814644,NoCurbRamp,410,400877098,West Oradell,3.0,False,3,1,0,POINT (-74.04623 40.95374)
50,814637,NoCurbRamp,385,61313120,West Oradell,3.0,False,4,0,1,POINT (-74.04551 40.95789)
51,813045,NoCurbRamp,136,11577473,East Oradell,4.0,False,1,0,0,POINT (-74.00506 40.95242)
55,818053,NoCurbRamp,11,11565017,Central Oradell,NaN,False,3,0,1,POINT (-74.03101 40.95455)
...,...,...,...,...,...,...,...,...,...,...,...
5772,816171,NoCurbRamp,29,11566560,South Oradell,5.0,False,2,0,0,POINT (-74.03117 40.94645)
5796,814577,NoCurbRamp,347,11587994,West Oradell,5.0,False,2,2,0,POINT (-74.04712 40.96309)
5801,814632,NoCurbRamp,343,11587980,West Oradell,5.0,False,1,1,0,POINT (-74.04069 40.95926)
5816,816166,NoCurbRamp,3,11564127,South Oradell,4.0,False,5,1,0,POINT (-74.04323 40.94723)


In [30]:
#export missing curb ramps to json
missing_curb_ramps_m.to_file('data/points_full.geojson', driver='GeoJSON')

In [14]:
#export to geojson
s_a_389.to_file("data/street.geojson", driver='GeoJSON')


In [70]:
base = s_b.explore( tiles = "CartoDB dark matter", color = 'green')
base